In [1]:
import dataverk as dv

import json
import pandas as pd
import ipywidgets as widgets

import plotly.offline as py
import plotly.graph_objs as go
import plotly.io as plio
import plotly_express as px

/Users/B149899/anaconda3/envs/dv/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/B149899/anaconda3/envs/dv/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# Get data

In [2]:
hovedtall = 'https://www.nav.no/no/NAV+og+samfunn/Statistikk/Arbeidssokere+og+stillinger+-+statistikk/Hovedtall+om+arbeidsmarkedet/_attachment/548327?_download=true&_ts=16a78e99880'
sesongjusterte_hovedtall = 'https://www.nav.no/no/NAV+og+samfunn/Statistikk/Arbeidssokere+og+stillinger+-+statistikk/Hovedtall+om+arbeidsmarkedet/_attachment/548327?_download=true&_ts=16a78e99880'
helt_ledige = 'https://www.nav.no/no/NAV+og+samfunn/Statistikk/Arbeidssokere+og+stillinger+-+statistikk/Hovedtall+om+arbeidsmarkedet/_attachment/548329?_download=true&_ts=16a78e99880'


In [3]:
df_hl = pd.read_excel(hovedtall, sheet_name='6. Helt ledige tidsserie måned', header=7, nrows=10)
df_hl.rename(columns={ df_hl.columns[0]: "År" }, inplace=True)
df_hl_m = pd.melt(df_hl, id_vars=['År'])
df_hl_m.columns=['År', 'Måned', 'Antall ledige']
df_hl_m.head(5)

,År,Måned,Antall ledige
0,2019,Januar,71148.0
1,2018,Januar,71849.0
2,2017,Januar,88238.0
3,2016,Januar,93284.0
4,2015,Januar,82577.0


In [4]:
df_hlp = pd.read_excel(hovedtall, sheet_name='7. Helt ledige prosent av arbei', header=7, nrows=10)
df_hlp.rename(columns={ df_hlp.columns[0]: "År" }, inplace=True)
df_hlp_m = pd.melt(df_hlp, id_vars=['År'])
df_hlp_m.columns=['År', 'Måned', 'Antall ledige']
df_hlp_m.head(5)

,År,Måned,Antall ledige
0,2019,Januar,2.6
1,2018,Januar,2.6
2,2017,Januar,3.2
3,2016,Januar,3.4
4,2015,Januar,3.0


# Create views

## Line chart

In [5]:
import datetime
now = datetime.datetime.now()

def getLine(år):
    if (år == now.year):
        return dict(color = ('#C30000'),width = 2)
    else:
        return dict(color = ('#3E3832'),width = 2)
    
def getOpacity(år):
    if (år == now.year):
        return 1
    else:
        return 1/(1 + now.year - år)

data = []

for år in df_hl_m['År'].unique():
    df = df_hl_m[df_hl_m['År'] == år]
    trace = go.Scatter(
        x = df['Måned'],
        y = df['Antall ledige'],
        name=str(år),
        line = getLine(år),
        opacity = getOpacity(år)
    )
    data.append(trace)

layout = go.Layout(title='Antall helt ledige')
fig_1 = go.Figure(data,layout)

go.FigureWidget(data,layout)

FigureWidget({
    'data': [{'line': {'color': '#C30000', 'width': 2},
              'name': '2019',
         …

In [6]:
data = []

for år in df_hlp_m['År'].unique():
    df = df_hlp_m[df_hlp_m['År'] == år]
    trace = go.Scatter(
        x = df['Måned'],
        y = df['Antall ledige'],
        name=str(år),
        line = getLine(år),
        opacity = getOpacity(år)
    )
    data.append(trace)

layout = go.Layout(title='Antall helt ledige i prosent av arbeidsstyrken')
fig_2 = go.Figure(data,layout)

go.FigureWidget(data,layout)

FigureWidget({
    'data': [{'line': {'color': '#C30000', 'width': 2},
              'name': '2019',
         …

## Map view

In [7]:
url = "https://www.nav.no/no/NAV+og+samfunn/Statistikk/Arbeidssokere+og+stillinger+-+statistikk/Hovedtall+om+arbeidsmarkedet/_attachment/546920?_download=true&_ts=169c36f05f8"
df_k = pd.read_csv(url, sep=';', encoding='latin1', dtype={'År-måned': object,'Kommunenr': object })

In [8]:
df_k.head()

,År-måned,Fylkenr,Fylkenavn,Kommunenr,Kommunenavn,Antall helt ledige,Helt ledige i prosent av arbeidsstyrken
0,201903,--,Ukjent,-1,Ukjent,5,NaN
1,201903,01,Østfold,0101,Halden,460,"3,2"
2,201903,01,Østfold,0104,Moss,479,"3,1"
3,201903,01,Østfold,0105,Sarpsborg,901,"3,5"
4,201903,01,Østfold,0106,Fredrikstad,1338,"3,4"


In [9]:
df_mk = pd.read_csv("https://raw.githubusercontent.com/sketl/dataset/master/meta/kommune_geo.tsv", sep='\t', dtype={'kommunenr': object })

df_km = pd.merge(df_k, df_mk, left_on='Kommunenr', right_on='kommunenr', how='left')
df_mkl = df_km[['kommune','År-måned','Antall helt ledige','Helt ledige i prosent av arbeidsstyrken','lat','lon']]
df_mkl = df_mkl[(df_mkl['kommune'].notna()) & (df_mkl['Antall helt ledige'] != '*')].copy()

def getCoordinates(row):
    return ["{0:.4f}".format(row['lat']), "{0:.4f}".format(row['lon'])]

df_mkl['coordinates'] = df_mkl.apply(getCoordinates, axis=1)
df_mkl.head()

,kommune,År-måned,Antall helt ledige,Helt ledige i prosent av arbeidsstyrken,lat,lon,coordinates
1,Halden,201903,460,"3,2",59.12258,11.38709,"[59.1226, 11.3871]"
2,Moss,201903,479,"3,1",59.43436,10.65968,"[59.4344, 10.6597]"
3,Sarpsborg,201903,901,"3,5",59.28148,11.11167,"[59.2815, 11.1117]"
4,Fredrikstad,201903,1338,"3,4",59.21759,10.93787,"[59.2176, 10.9379]"
5,Hvaler,201903,45,"2,1",59.02809,11.03110,"[59.0281, 11.0311]"


In [10]:
df_mkl.columns=['name','periode','sum','andel','lat', 'lon','coordinates']
df_mkl['andel'] = df_mkl['andel'].apply(lambda x: float(x.replace(',','.')))
df_mkl['sum'] = df_mkl['sum'].apply(lambda x: float(x))
df_mkl.head()

,name,periode,sum,andel,lat,lon,coordinates
1,Halden,201903,460.0,3.2,59.12258,11.38709,"[59.1226, 11.3871]"
2,Moss,201903,479.0,3.1,59.43436,10.65968,"[59.4344, 10.6597]"
3,Sarpsborg,201903,901.0,3.5,59.28148,11.11167,"[59.2815, 11.1117]"
4,Fredrikstad,201903,1338.0,3.4,59.21759,10.93787,"[59.2176, 10.9379]"
5,Hvaler,201903,45.0,2.1,59.02809,11.03110,"[59.0281, 11.0311]"


In [12]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig_map = px.scatter_mapbox(df_mkl, lat="lat", lon="lon", color="andel", size="sum", hover_name="name",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=50, zoom=2.5, animation_frame="periode")
fig_map

In [34]:
# prepare data for choropleth map

def getValue(string):
    try:
        return int(string)
    except ValueError:
        return 0

utv = df_k[df_k['År-måned']=='201903'][['Fylkenavn','Antall helt ledige']].copy()
utv['Antall helt ledige'] = utv['Antall helt ledige'].map(getValue)
utv.columns=['name', 'value']
antall_pr_fylke = utv.groupby('name').sum().reset_index()
json.loads(antall_pr_fylke.to_json(orient='records'))

[{'name': 'Akershus', 'value': 6719},
 {'name': 'Aust-Agder', 'value': 1463},
 {'name': 'Buskerud', 'value': 3641},
 {'name': 'Finnmark Finnmárku', 'value': 1147},
 {'name': 'Hedmark', 'value': 2231},
 {'name': 'Hordaland', 'value': 6906},
 {'name': 'Møre og Romsdal', 'value': 3230},
 {'name': 'Nordland', 'value': 2624},
 {'name': 'Oppland', 'value': 1806},
 {'name': 'Oslo', 'value': 10133},
 {'name': 'Rogaland', 'value': 6131},
 {'name': 'Sogn og Fjordane', 'value': 968},
 {'name': 'Telemark', 'value': 2412},
 {'name': 'Troms Romsa', 'value': 1573},
 {'name': 'Trøndelag', 'value': 4929},
 {'name': 'Ukjent', 'value': 5},
 {'name': 'Vest-Agder', 'value': 2213},
 {'name': 'Vestfold', 'value': 3756},
 {'name': 'Østfold', 'value': 4340},
 {'name': 'Øvrige områder', 'value': 12}]

In [129]:
# prepare data for marker map

def parseint(value):
    try:
        int(value)
        return int(value)
    except:
        return None
    
df_mf = pd.read_csv("https://raw.githubusercontent.com/sketl/dataset/master/meta/fylke_geo.tsv", sep='\t', dtype={'kommunenr': object })
df_k["fylkenr_int"] = df_k["Fylkenr"].map(parseint)
df_k["Antall helt ledige int"] = df_k["Antall helt ledige"].map(parseint)


df_mm = pd.merge(df_k, df_mf, left_on='fylkenr_int', right_on='fylkenr', how='left')
df_mm = df_mm[['Fylkenavn','År-måned','Antall helt ledige int','lat','lon']]
df_mm = df_mm[(df_mm['Fylkenavn'].notna()) & (df_mm['lat'].notna())].copy()

,Fylkenavn,År-måned,Antall helt ledige int,lat,lon
1,Østfold,201903,460.0,59.27525,11.10596
2,Østfold,201903,479.0,59.27525,11.10596
3,Østfold,201903,901.0,59.27525,11.10596
4,Østfold,201903,1338.0,59.27525,11.10596
5,Østfold,201903,45.0,59.27525,11.10596


In [90]:
df_mm_g = df_mm.groupby(['Fylkenavn','År-måned','lat','lon']).sum().reset_index()

In [115]:
def getLatLon(row):
    return [round(float(row['lon']),2), round(float(row['lat']),2)]

df_mm_g['lat_lon'] = df_mm_g.apply(getLatLon, axis=1)
df_mm_g_f = df_mm_g[['Fylkenavn', 'Antall helt ledige int','lat_lon']]
df_mm_g_f.columns=['name', 'value', 'coordinates']
json.loads(df_mm_g_f.to_json(orient='records'))

# Create datapackage

In [117]:
readme = """
# Arbeidsmarkedet hovedtall

NAV publiserer sentral statistikk om arbeidsmarkedet hver måned.

kilde: [NAV](https://www.nav.no/no/NAV+og+samfunn/Statistikk/Arbeidssokere+og+stillinger+-+statistikk/Hovedtall+om+arbeidsmarkedet)
"""

metadata = {
'store': 'gs',
'repo': 'https://raw.githubusercontent.com/datasett/nav-examples/master',
'readme': readme,
'license':'MIT',
'accessRights':'Open',
'author': 'paul.bencze@nav.no',
'description':'Arbeidsmarkedet hovedtall',
'title':'Arbeidsmarkedet hovedtall',
'name':'Arbeidsmarkedet hovedtall',
'source':'NAV',
'keywords':['arbeidsmarkedet'],
'provenance':'NAV',
'publisher': 'NAV', 
'bucket': 'nav-opendata'
}

In [118]:
dp = dv.Datapackage(metadata)

## Add views to datapackage

In [119]:
description = '''
Merk: Brudd i serien. I november 2018 startet NAV med ny registreringsløsning for alle som registrerer seg som arbeidssøkere på nav.no. Arbeidssøkere kan i ny løsning begynne med egne jobbsøkingsaktiviteter med en gang og bli raskere disponible for arbeidsmarkedet. Før dette (i perioden 2005-2018) måtte arbeidssøkere vente til registreringen ble godkjent av en saksbehandler. Registreringen er nå mindre omfattende og fører til at det går kortere tid fra arbeidssøkerne registrerer seg til de kommer inn i statistikken. Dette gir en økning i antallet helt ledige som ellers ikke ville vært der og brudd i statistikken. Samlet brudd estimeres til 6 prosent flere helt ledige. Effekt av bruddet er estimert tilbake til 2005 og brukt til å lage brudd- og sesongjustert tidsserier for beholdning arbeidssøkere. Justeringen er utført ved å prekorrigere tidligere publiserte tall og deretter sesongjustere.
Bruddet skjedde gradvis i november, desember 2018 og januar 2019. 
'''

dp.add_view(
    spec_type='plotly',
    name='Antall helt ledige',
    title='Antall helt ledige',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=plio.to_json(fig_1)
)

In [120]:
description = '''

'''

dp.add_view(
    spec_type='plotly',
    name='Antall helt ledige i prosent',
    title='Antall helt ledige i prosent',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=plio.to_json(fig_2)
)

In [130]:
description = '''

'''

dp.add_view(
    spec_type='plotly',
    name='Antall helt ledige i prosent',
    title='Antall helt ledige i prosent',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=plio.to_json(fig_map)
)

In [121]:
desc = '''
### Antall ledige pr fylke
'''

dp.add_view(
    spec_type='choropleth',
    name='Antall ledige pr fylke (simple)',
    title='Antall ledige pr fylke',
    description =desc,
    resources = '',
    spec= {'map': 'norway/fylker-simple.topo.json',
           'data': json.loads(antall_pr_fylke.to_json(orient='records'))}
)

In [122]:
desc = '''
### Antall ledige pr fylke
'''

"""
dp.add_view(
    spec_type='choropleth',
    name='Antall ledige pr fylke',
    title='Antall ledige pr fylke',
    description =desc,
    resources = '',
    spec= {'map': 'norway/fylker-simple.topo.json',
           'data': antall_pr_fylke.to_json(orient='records')}
)
"""

"\ndp.add_view(\n    spec_type='choropleth',\n    name='Antall ledige pr fylke',\n    title='Antall ledige pr fylke',\n    description =desc,\n    resources = '',\n    spec= {'map': 'norway/fylker-simple.topo.json',\n           'data': antall_pr_fylke.to_json(orient='records')}\n)\n"

In [123]:
desc = '''
### Antall ledige pr fylke (markers)
'''

dp.add_view(
    spec_type='topojson',
    name='Antall ledige pr fylke',
    title='Antall ledige pr fylke',
    description =desc,
    resources = '',
    spec= { 'markers': json.loads(df_mm_g_f.to_json(orient='records')),
      'markerMinSize': 10,
      'markerMaxSize': 30,
   }
)

In [124]:
desc = '''
### Antall ledige pr kommune
'''

"""
dp.add_view(
    spec_type='choropleth',
    name='Antall ledige pr kommune',
    title='Antall ledige pr kommune',
    description =desc,
    resources = '',
    spec= {'map': 'norway/kommuner.topo.json',
           'data': antall_pr_fylke.to_json(orient='records')}
)
"""

"\ndp.add_view(\n    spec_type='choropleth',\n    name='Antall ledige pr kommune',\n    title='Antall ledige pr kommune',\n    description =desc,\n    resources = '',\n    spec= {'map': 'norway/kommuner.topo.json',\n           'data': antall_pr_fylke.to_json(orient='records')}\n)\n"

In [125]:
dp.add_view(
    spec_type='table',
    description = '',
    name='Antall helt ledige',
    title='Antall helt ledige',
    resources = 'Antall helt ledige'
)

In [126]:
dp.add_resource(df_hl,'Antall helt ledige','Antall helt ledige')
dp.add_resource(df_hlp,'Antall helt ledige i prosent','Antall helt ledige i prosent')

In [127]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/users/b149899/projects/binder/binder/keys/nav_opendata.json'

# Save data and publish datapackage

In [128]:
client = dv.Client()
client.publish(dp)

gs
2019-06-18T10:17:28.020751: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: json) written to https://storage.googleapis.com/nav-opendata/6b4b4e62a8f903003e849b863778ef8f/datapackage.json
2019-06-18T10:17:28.504358: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: csv) written to https://storage.googleapis.com/nav-opendata/6b4b4e62a8f903003e849b863778ef8f/resources/Antall%20helt%20ledige.csv
2019-06-18T10:17:28.985373: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: csv) written to https://storage.googleapis.com/nav-opendata/6b4b4e62a8f903003e849b863778ef8f/resources/Antall%20helt%20ledige%20i%20prosent.csv
2019-06-18T10:17:30.725814: B149899 <class 'dataverk.connectors.elasticsearch.ElasticsearchConnector'>: Document 6b4b4e62a8f903003e849b863778ef8f of type odata indexed to elastic index: odata.
